# Redis Data Integration Demo
This notebook sets up Change Data Capture (CDC) environment with roles, permissions, and sample data for demoing Redis Data Integration.

In [ ]:
// Role and permissions setup

CREATE OR REPLACE ROLE riotx_cdc
COMMENT = 'minimum cdc role for riotx';

-- Grant warehouse access for compute operations
GRANT USAGE, OPERATE ON WAREHOUSE compute_wh TO ROLE riotx_cdc;

CREATE DATABASE IF NOT EXISTS tb_101;
USE DATABASE tb_101;

CREATE OR REPLACE SCHEMA tb_101.raw_pos;
CREATE OR REPLACE SCHEMA tb_101.raw_pos_cdc;

-- Grant basic database and schema access to the CDC role
GRANT USAGE ON DATABASE tb_101 TO ROLE riotx_cdc;
GRANT USAGE ON SCHEMA tb_101.raw_pos TO ROLE riotx_cdc;
GRANT USAGE ON SCHEMA tb_101.raw_pos_cdc TO ROLE riotx_cdc;

GRANT SELECT ON FUTURE TABLES IN SCHEMA tb_101.raw_pos_cdc TO ROLE riotx_cdc;
GRANT CREATE TABLE ON SCHEMA tb_101.raw_pos_cdc TO ROLE riotx_cdc;
GRANT CREATE STREAM ON SCHEMA tb_101.raw_pos_cdc TO ROLE riotx_cdc;
GRANT SELECT ON FUTURE STREAMS IN SCHEMA tb_101.raw_pos_cdc TO ROLE riotx_cdc;

CREATE OR REPLACE USER riotx_cdc
    DEFAULT_ROLE = 'riotx_cdc'
    DEFAULT_WAREHOUSE = 'compute_wh'
    PASSWORD = '{{PASSWORD}}';

-- Assign roles to enable proper access hierarchy
GRANT ROLE riotx_cdc TO ROLE accountadmin;
GRANT ROLE riotx_cdc TO USER riotx_cdc;
GRANT ROLE accountadmin TO USER riotx_cdc;

In [ ]:
// Create and populate table

CREATE OR REPLACE FILE FORMAT tb_101.public.csv_ff type = 'csv';

CREATE OR REPLACE STAGE tb_101.public.s3load
    COMMENT = 'Quickstarts S3 Stage Connection'
    URL = 's3://sfquickstarts/frostbyte_tastybytes/'
    FILE_FORMAT = tb_101.public.csv_ff;

CREATE OR REPLACE TABLE tb_101.raw_pos.order_header
(
    order_id NUMBER(38,0),
    truck_id NUMBER(38,0),
    location_id FLOAT,
    customer_id NUMBER(38,0),
    discount_id VARCHAR(16777216),
    shift_id NUMBER(38,0),
    shift_start_time TIME(9),
    shift_end_time TIME(9),
    order_channel VARCHAR(16777216),
    order_ts TIMESTAMP_NTZ(9),
    served_ts VARCHAR(16777216),
    order_currency VARCHAR(3),
    order_amount NUMBER(38,4),
    order_tax_amount VARCHAR(16777216),
    order_discount_amount VARCHAR(16777216),
    order_total NUMBER(38,4)
);

-- Load data from S3 stage with size limit
COPY INTO tb_101.raw_pos.order_header
    FROM @tb_101.public.s3load/raw_pos/order_header/
    SIZE_LIMIT = 1000;

CREATE OR REPLACE TABLE tb_101.raw_pos.incremental_order_header LIKE tb_101.raw_pos.order_header;

-- Enable change tracking for CDC functionality
ALTER TABLE tb_101.raw_pos.INCREMENTAL_ORDER_HEADER SET CHANGE_TRACKING = TRUE;

-- Grant read access to both tables for the CDC role
GRANT SELECT ON TABLE tb_101.raw_pos.order_header TO ROLE riotx_cdc;
GRANT SELECT ON TABLE tb_101.raw_pos.incremental_order_header TO ROLE riotx_cdc;

-- Insert initial rows
INSERT INTO tb_101.raw_pos.incremental_order_header
    SELECT * FROM tb_101.raw_pos.order_header
    LIMIT 100 OFFSET 0;

-- Show first rows
SELECT * FROM tb_101.raw_pos.incremental_order_header limit 10;

In [ ]:
-- Insert additional data
INSERT INTO tb_101.raw_pos.incremental_order_header
   SELECT * FROM tb_101.raw_pos.order_header
   LIMIT 100 OFFSET 100;

SELECT COUNT(*) FROM tb_101.raw_pos.incremental_order_header;

In [ ]:
// Update a row

UPDATE tb_101.raw_pos.incremental_order_header SET truck_id=123456 WHERE order_id=4063900;

In [ ]:
// Delete a row

DELETE FROM tb_101.raw_pos.incremental_order_header where ORDER_ID=4063900;